configure mysql/python connection and prepare the databse for use

In [1]:
# import password from config file
from config import db_password

In [2]:
# import mysql.connector
import mysql.connector as connector

In [5]:
# connect MySQL with Python
connection = connector.connect(user='root', password = db_password)
print("Connection between MySQL and Python is established.\n")

Connection between MySQL and Python is established.



In [6]:
# create cursor
cursor = connection.cursor()
print("Cursor is created to communicate with the MySQL using Python.\n")

Cursor is created to communicate with the MySQL using Python.



In [7]:
# use the database
cursor.execute('USE little_lemon_db;')
print("The database little_lemon is set for use.\n")

The database little_lemon is set for use.



ESTABLISH POOL CONNECTION 

In [8]:
# import pool connection and Error from mysql.connector.pooling
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

In [9]:
# configure dbconfig
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": db_password
}

try:
    pool =MySQLConnectionPool(pool_name='pool_a', pool_size=2, **dbconfig)
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The connection pool size is: ', pool.pool_size)

except Error as er:
    print('Error: ', er.errno)
    print('Message: ', er.msg)

The connection pool is created with a name:  pool_a
The connection pool size is:  2


In [10]:
# get connection object from a pool
print('\nGetting connection from a pool ...\n')
connection1 = pool.get_connection()
print("'Connection1' object is: ", connection1)


Getting connection from a pool ...

'Connection1' object is:  <mysql.connector.pooling.PooledMySQLConnection object at 0x105eed040>


In [11]:
# create cursor object from a connection
cursor = connection1.cursor()

CREATE PROCEDURES

In [12]:
# CREATE PROCEDURE PeakHours
cursor.execute('DROP PROCEDURE IF EXISTS PeakHours;')
stored_procedure_peak_hours = ''' CREATE PROCEDURE PeakHours()
BEGIN
SELECT HOUR(BookingSlot) AS Booking_Hour, COUNT(HOUR(BookingSlot)) AS n_Bookings
FROM Bookings
GROUP BY HOUR(BookingSlot)
ORDER BY n_Bookings DESC;
END
'''


In [13]:
# execute the stored procedure
cursor.execute(stored_procedure_peak_hours)

In [14]:
# call the stored procedure
cursor.callproc('PeakHours')

# retrieve records in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# retrieve the column names
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# print the column names and dataset
print(columns)

#print the dataset
for row in dataset:
    print(row)


['Booking_Hour', 'n_Bookings']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


stored procedure GuestStatus

In [18]:
# create procedure GuestStatus
cursor.execute('DROP PROCEDURE IF EXISTS GuestStatus;')
stored_procedure_guest_status = ''' CREATE PROCEDURE GuestStatus()
BEGIN 
SELECT CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS 'Guest Name', 
e.Role AS Employee,

CASE 
WHEN e.Role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay'
WHEN e.Role = 'Head Chef' THEN 'Ready to serve'
WHEN e.Role = 'Assistant Chef' THEN 'Preparing Order' 
WHEN e.Role = 'Head Waiter' THEN 'Order Served' 
ELSE 'Pending'
END AS Status

FROM Bookings AS b
LEFT JOIN Employees AS e
ON b.EmployeeID = e.EmployeeID;

END

'''

In [19]:
# execute the stored procedure
cursor.execute(stored_procedure_guest_status)


In [20]:
# call the stored procedure
cursor.callproc('GuestStatus')

# retrieve records in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# retrieve the column names
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# print the column names and dataset
print(columns)

#print the dataset
for row in dataset:
    print(row)

# close connection
connection1.close()


['Guest Name', 'Employee', 'Status']
('Anna Iversen', 'Manager', 'Ready to pay')
('Joakim Iversen', 'Manager', 'Ready to pay')
('Vanessa McCarthy', 'Head Chef', 'Ready to serve')
('Marcos Romero', 'Assistant Chef', 'Preparing Order')
('Hiroki Yamane', 'Assistant Manager', 'Ready to pay')
('Diana Pinto', 'Head Waiter', 'Order Served')


create connection for pool_b

In [21]:
# configure connection for pool_b using try-except block
try:
    pool = MySQLConnectionPool(pool_name='pool_b', pool_size=2, **dbconfig)
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The connection pool size is: ', pool.pool_size)

except Error as er:
    print('Error: ', er.errno)
    print('Message: ', er.msg)


The connection pool is created with a name:  pool_b
The connection pool size is:  2


ADD connections for three guests and their data

In [22]:
# get connection for the first guest from a pool
connection1 = pool.get_connection()
# create cursor object from a connection for the 1st guest
cursor1 = connection1.cursor()

# insert data into Bookings table for the guest N1
booking1 = ''' INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,'Anees','Java','18:00:00',6);'''

# execute booking1
cursor1.execute(booking1)

In [23]:
connection1.commit()

In [24]:
# connect 2nd guest
connection2 = pool.get_connection()
cursor2 = connection2.cursor()
booking2 = ''' INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);'''

cursor2.execute(booking2)

In [25]:
connection2.commit()

In [30]:
# adding a new connection for the 3rd guest with try-except block
import mysql.connector as connector
try:
    connection3 = pool.get_connection()
    print('The guest is connected')

except:
    print('Adding a new connection in the pool')

    # create a new connection
    connection=connector.connect(user= 'root', password= db_password)
    # add connection into the pool
    pool.add_connection(cnx=connection)
    print('A new connection is added into the pool. \n')
    
# connect 3rd guest
cursor3 = connection3.cursor()
booking3 = ''' INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);'''
cursor3.execute(booking3)


Adding a new connection in the pool
A new connection is added into the pool. 



In [31]:
connection3.commit()
print(''' A new connection is added in the "Bookings" table. ''')

 A new connection is added in the "Bookings" table. 


In [32]:
# You can only return two connections back to the pool as the pool_size=2.
#  Close all  connections and use try-except to print the pool error if the pool is already full.
for connection in [connection1, connection2, connection3]:
    try:
        connection.close()
        print('Connection returned to the pool successfully.')
    except Error as er:
        print("\nConnection can't be returned to the pool.")
        print('Error message: ', er.msg)
              

Connection returned to the pool successfully.

Connection can't be returned to the pool.
Error message:  Failed adding connection; queue is full

Connection can't be returned to the pool.
Error message:  Failed adding connection; queue is full


 create report on employee highest salary and number of bookings

In [53]:
# Get a connection from pool_a and create a cursor object to communicate with the database. 
print("Getting a connection from the pool.")
connection = pool.get_connection()
print("""The object "connection" is created with a connection link from the pool_a""")
print("""Creating a cursor object to communicate with the database.""")
cursor=connection.cursor()
print("""The cursor object "cursor" is created.""")

# The name and EmployeeID of the Little Lemon manager.
cursor.execute("SELECT EmployeeID, Name FROM Employees WHERE Role = 'Manager';")
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

Getting a connection from the pool.
The object "connection" is created with a connection link from the pool_a
Creating a cursor object to communicate with the database.
The cursor object "cursor" is created.
('EmployeeID', 'Name')
(1, 'Mario Gollini')


In [49]:
# The name and role of the employee who receives the highest salary.
cursor.execute(''' 
               SELECT Name, EmployeeID
               FROM Employees
               ORDER BY Annual_Salary DESC
               LIMIT 1;
               ''')
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

('Name', 'EmployeeID')
('Mario Gollini', 1)


In [54]:
# The number of guests booked between 18:00:00 and 20:00:00.
cursor.execute('''
               SELECT COUNT(BookingID) AS n_Guests_between_18_20_hrs 
               FROM Bookings
               WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00'; ''')
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

('n_Guests_between_18_20_hrs',)
(8,)


In [55]:
# The full name and the BookingId of each guest waiting to be seated with 
# the receptionist in sorted order with respect to their BookingSlot.
cursor.execute(''' 
               SELECT b.BookingID AS ID, CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS 'Guest Name',
               e.Role AS Employee 
               FROM Bookings AS b
               LEFT JOIN Employees AS e
               ON b.EmployeeID = e.EmployeeID
               WHERE e.Role = 'Receptionist' 
               ORDER BY BookingSlot DESC; ''')
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

('ID', 'Guest Name', 'Employee')
(9, 'Jay Kon', 'Receptionist')
(10, 'Jay Kon', 'Receptionist')
(8, 'Bald Vin', 'Receptionist')
(7, 'Anees Java', 'Receptionist')


TASK 4 EXERCISE 3

In [60]:
# Create a stored procedure named BasicSalesReport. 
cursor.execute('DROP PROCEDURE IF EXISTS BasicSalesReport;')

stored_procedure_basic_sales_report = ''' CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT 
SUM(BillAmount) AS Total_Sales,
AVG(BillAmount) AS Average_Sales,
MIN(BillAmount) AS Minimum_Sales,
MAX(BillAmount) AS Maximum_Sales
FROM Orders;
END
'''

cursor.execute(stored_procedure_basic_sales_report)
#  call the stored procedure
cursor.callproc('BasicSalesReport')

# retrieve records in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# retrieve the column names
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# print the column names and dataset
print("Today's Sales Report")
for data in dataset:
    print('\t', columns[0], ':', data[0])
    print('\t', columns[1], ':', data[1])
    print('\t', columns[2], ':', data[2])
    print('\t', columns[3], ':', data[3])


Today's Sales Report
	 Total_Sales : 243
	 Average_Sales : 48.6000
	 Minimum_Sales : 37
	 Maximum_Sales : 86


Task 5

In [69]:
connection = pool.get_connection()
cursor = connection.cursor(buffered=True)

select_upcoming_bookings = ''' 
SELECT 
b.BookingSlot, 
CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS Guest_Name,
e.Name AS Employee_Name,
e.Role AS Role
FROM Bookings AS b
INNER JOIN Employees AS e
ON b.EmployeeID = e.EmployeeID
ORDER BY b.BookingSlot ASC;
'''

cursor.execute(select_upcoming_bookings)
results = cursor.fetchmany(size=3)

for result in results:
    print('\nBookingSlot', result[0])
    print('\tGuest_Name', result[1])
    print('\tAssigned to: ', result[2], '[{}]'.format(result[3]))




BookingSlot 15:00:00
	Guest_Name Vanessa McCarthy
	Assigned to:  Giorgos Dioudis [Head Chef]

BookingSlot 17:30:00
	Guest_Name Marcos Romero
	Assigned to:  Fatma Kaya [Assistant Chef]

BookingSlot 18:00:00
	Guest_Name Anees Java
	Assigned to:  John Millar [Receptionist]


In [63]:
connection.close()